# Queries

In [2]:
import pandas as pd
from ssh_pymongo import MongoSession

In [14]:
session = MongoSession(
    host='mesiin592023-00057.westeurope.cloudapp.azure.com',
    uri='mongodb://mesiin592023-00057:30000',
    user='administrateur',
    password='SuperPassword!1'
)

In [15]:
db = session.connection['employee_db']

In [16]:
employees = db["employees"]

In [ ]:
df = pd.DataFrame(employees.find())

In [17]:
# Ru1
pipeline_ru1 = [
    {"$unwind": "$departments"},
    {"$group": {
        "_id": {
            "department": "$departments.dept_name",
            "gender": "$gender"
        },
        "count": {"$sum": 1}
    }},
    {"$project": {
        "department": "$_id.department",
        "gender": "$_id.gender",
        "count": 1,
        "_id": 0
    }}
]
result_ru1 = list(db.employees.aggregate(pipeline_ru1))

In [18]:
query_ru2 = {
    "birth_date": {"$gt": "1963-01-01"}
}
projection_ru2 = {
    "_id": 0,
    "first_name": 1,
    "last_name": 1,
    "birth_date": 1,
    "departments.dept_name": 1
}
result_ru2 = list(db.employees.find(query_ru2, projection_ru2))

In [19]:
# Ru3
query_ru3 = {
    "titles": {
        "$not": {
            "$elemMatch": {
                "to_date": "9999-01-01"
            }
        }
    }
}
projection_ru3 = {
    "_id": 0,
    "emp_no": 1,
    "first_name": 1,
    "last_name": 1,
    "titles": 1,
    "hire_date": 1,
    "salaries": 1
}
result_ru3 = list(db.employees.find(query_ru3, projection_ru3))

In [ ]:
# # Ru4
# pipeline_ru4 = [
#     {"$match": {"departments.is_manager": True}},
#     {"$project": {
#         "first_name": 1,
#         "last_name": 1,
#         "titles": {
#             "$filter": {
#                 "input": "$titles",
#                 "as": "title",
#                 "cond": {
#                     "$eq": [False, {"$in": ["$$title.from_date", "$departments.from_date"]}]
#                 }
#             }
#         }
#     }}
# ]
# result_ru4 = list(db.emp.aggregate(pipeline_ru4))

In [20]:
# Ru6
pipeline_ru6 = [
    {"$match": {"departments.dept_name": "Finance"}},
    {"$unwind": "$salaries"},
    {"$match": {"salaries.to_date": "9999-01-01"}},
    {"$sort": {"salaries.salary": -1}},
    {"$limit": 3},
    {"$project": {
        "_id": 0,
        "first_name": 1,
        "last_name": 1,
        "salary": "$salaries.salary"
    }}
]
result_ru6 = list(db.employees.aggregate(pipeline_ru6))

In [21]:
pipeline_rda1 = [
    {"$match": {"gender": "F", "departments.is_manager": True}},
    {"$project": {
        "hireYear": {"$year": {"$dateFromString": {"dateString": "$hire_date"}}}
    }},
    {"$group": {
        "_id": "$hireYear",
        "count": {"$sum": 1}
    }},
    {"$sort": {"_id": 1}}
]
result_rda1 = list(db.employees.aggregate(pipeline_rda1))

In [22]:
# Rda4
pipeline_rda4 = [
    {"$unwind": "$departments"},
    {"$unwind": "$salaries"},
    {"$group": {
        "_id": "$departments.dept_name",
        "totalEmployees": {"$sum": 1},
        "averageSalary": {"$avg": "$salaries.salary"},
        "minSalary": {"$min": "$salaries.salary"},
        "maxSalary": {"$max": "$salaries.salary"}
    }},
    {"$project": {
        "department": "$_id",
        "totalEmployees": 1,
        "averageSalary": 1,
        "minSalary": 1,
        "maxSalary": 1
    }},
    {"$sort": {"department": 1}}
]
result_rda4 = list(db.employees.aggregate(pipeline_rda4))

In [23]:
# Rda5
pipeline_rda5 = [
    {"$unwind": "$departments"},
    {"$match": {"departments.dept_name": {"$in": ["Sales", "Marketing"]}}},
    {"$unwind": "$salaries"},
    {"$match": {"salaries.to_date": "9999-01-01"}},
    {"$group": {
        "_id": "$departments.dept_name",
        "averageSalary": {"$avg": "$salaries.salary"}
    }},
    {"$project": {"_id": 0, "department": "$_id", "averageSalary": 1}}
]
result_rda5 = list(db.employees.aggregate(pipeline_rda5))

In [24]:
# Rda6
query_rda6 = {
    "departments.dept_name": "Production"
}
projection_rda6 = {
    "_id": 0,
    "emp_no": 1,
    "first_name": 1,
    "last_name": 1,
    "birth_date": 1,
    "hire_date": 1,
    "gender": 1
}
result_rda6 = list(db.employees.find(query_rda6, projection_rda6))

In [ ]:
# import time
# queries = {
#     "ru1": {"method": db.employees.aggregate, "args": (pipeline_ru1)},
#     "ru2": {"method": db.employees.find, "args": (query_ru2, projection_ru2)},
#     "ru3": {"method": db.employees.find, "args": (query_ru3, projection_ru3)},
#     "ru6": {"method": db.employees.aggregate, "args": (pipeline_ru6)},
#     "rda1": {"method": db.employees.aggregate, "args": (pipeline_rda1)},
#     "rda4": {"method": db.employees.aggregate, "args": (pipeline_rda4)},
#     "rda5": {"method": db.employees.aggregate, "args": (pipeline_rda5)},
#     "rda6": {"method": db.employees.find, "args": (query_rda6, projection_rda6)}
# }


# for query in queries:
#     times = []
#     for _ in range(10):
#         start_time = time.time()
#         query["method"]
#         end_time = time.time()
#         times.append(end_time - start_time)
#     # Remove the max and min
#     times.remove(max(times))
#     times.remove(min(times))
#     # Calculate the average
#     avg_time = sum(times) / len(times)
#     print(f"Average execution time for {query}: {avg_time} seconds")

In [ ]:
# queries = {
#     "ru1": {"method": db.employees.aggregate, "args": (pipeline_ru1)},
#     "ru2": {"method": db.employees.find, "args": (query_ru2, projection_ru2)},
#     "ru3": {"method": db.employees.find, "args": (query_ru3, projection_ru3)},
#     "ru6": {"method": db.employees.aggregate, "args": (pipeline_ru6)},
#     "rda1": {"method": db.employees.aggregate, "args": (pipeline_rda1)},
#     "rda4": {"method": db.employees.aggregate, "args": (pipeline_rda4)},
#     "rda5": {"method": db.employees.aggregate, "args": (pipeline_rda5)},
#     "rda6": {"method": db.employees.find, "args": (query_rda6, projection_rda6)}
# }

# for query_name, query_details in queries.items():
#     times = []
#     for _ in range(10):
#         start_time = time.time()
#         # Execute the query
#         query_details["method"]
#         end_time = time.time()
#         times.append(end_time - start_time)
#     # Remove the max and minzd
#     times.remove(max(times))
#     times.remove(min(times))
#     # Calculate the average
#     avg_time = sum(times) / len(times)
#     print(f"Average execution time for {query_name}: {avg_time} seconds")

In [25]:
import timeit

def ru1():
    db.employees.aggregate(pipeline_ru1)

def ru2():
    db.employees.find(query_ru2, projection_ru2)

def ru3():
    db.employees.find(query_ru3, projection_ru3)

def ru6():
    db.employees.aggregate(pipeline_ru6)

def rda1():
    db.employees.aggregate(pipeline_rda1)

def rda4():
    db.employees.aggregate(pipeline_rda4)

def rda5():
    db.employees.aggregate(pipeline_rda5)

def rda6():
    db.employees.find(query_rda6, projection_rda6)

# 1 shard

In [26]:
queries = [ru1, ru2, ru3, ru6, rda1, rda4, rda5, rda6]

execution_times_1shard = {}

for query in queries:
    times = timeit.repeat(query, repeat=12, number=1)

    times.remove(max(times))
    times.remove(min(times))

    execution_times_1shard[query.__name__] = times

    average_time = sum(times) / len(times)

    print(f"Le temps d'exécution moyen pour {query.__name__} est {average_time} secondes.")

Le temps d'exécution moyen pour ru1 est 0.7851978099439293 secondes.
Le temps d'exécution moyen pour ru2 est 9.960075840353965e-06 secondes.
Le temps d'exécution moyen pour ru3 est 9.379955008625984e-06 secondes.
Le temps d'exécution moyen pour ru6 est 0.46284552006982266 secondes.
Le temps d'exécution moyen pour rda1 est 0.3349408499896526 secondes.
Le temps d'exécution moyen pour rda4 est 3.044426879985258 secondes.
Le temps d'exécution moyen pour rda5 est 2.2756212800275533 secondes.
Le temps d'exécution moyen pour rda6 est 9.600073099136353e-06 secondes.


# 2 shards

In [28]:
queries = [ru1, ru2, ru3, ru6, rda1, rda4, rda5, rda6]

execution_times_2shard = {}

for query in queries:
    times = timeit.repeat(query, repeat=12, number=1)

    times.remove(max(times))
    times.remove(min(times))

    execution_times_2shard[query.__name__] = times

    average_time = sum(times) / len(times)

    print(f"Le temps d'exécution moyen pour {query.__name__} est {average_time} secondes.")

Le temps d'exécution moyen pour ru1 est 0.4629561699461192 secondes.
Le temps d'exécution moyen pour ru2 est 1.0440032929182052e-05 secondes.
Le temps d'exécution moyen pour ru3 est 9.029917418956756e-06 secondes.
Le temps d'exécution moyen pour ru6 est 0.2881730099208653 secondes.
Le temps d'exécution moyen pour rda1 est 0.18644522009417414 secondes.
Le temps d'exécution moyen pour rda4 est 1.7745569300372153 secondes.
Le temps d'exécution moyen pour rda5 est 1.3200086600147187 secondes.
Le temps d'exécution moyen pour rda6 est 5.207001231610775e-05 secondes.


# 3 shards

In [29]:
queries = [ru1, ru2, ru3, ru6, rda1, rda4, rda5, rda6]

execution_times_3shard = {}

for query in queries:
    times = timeit.repeat(query, repeat=12, number=1)

    times.remove(max(times))
    times.remove(min(times))

    execution_times_3shard[query.__name__] = times

    average_time = sum(times) / len(times)

    print(f"Le temps d'exécution moyen pour {query.__name__} est {average_time} secondes.")

Le temps d'exécution moyen pour ru1 est 0.37240829998627306 secondes.
Le temps d'exécution moyen pour ru2 est 1.0220054537057877e-05 secondes.
Le temps d'exécution moyen pour ru3 est 8.87005589902401e-06 secondes.
Le temps d'exécution moyen pour ru6 est 0.2301679900381714 secondes.
Le temps d'exécution moyen pour rda1 est 0.1484525999519974 secondes.
Le temps d'exécution moyen pour rda4 est 1.3736435499973596 secondes.
Le temps d'exécution moyen pour rda5 est 1.0246574100106955 secondes.
Le temps d'exécution moyen pour rda6 est 9.24007035791874e-06 secondes.


# 4 shards

In [30]:
queries = [ru1, ru2, ru3, ru6, rda1, rda4, rda5, rda6]

execution_times_4shard = {}

for query in queries:
    times = timeit.repeat(query, repeat=12, number=1)

    times.remove(max(times))
    times.remove(min(times))

    execution_times_4shard[query.__name__] = times

    average_time = sum(times) / len(times)

    print(f"Le temps d'exécution moyen pour {query.__name__} est {average_time} secondes.")

Le temps d'exécution moyen pour ru1 est 0.2951745401602238 secondes.
Le temps d'exécution moyen pour ru2 est 9.569991379976273e-06 secondes.
Le temps d'exécution moyen pour ru3 est 9.020045399665833e-06 secondes.
Le temps d'exécution moyen pour ru6 est 0.19739240002818406 secondes.
Le temps d'exécution moyen pour rda1 est 0.11887439005076886 secondes.
Le temps d'exécution moyen pour rda4 est 1.0771062500309199 secondes.
Le temps d'exécution moyen pour rda5 est 0.8096781999804079 secondes.
Le temps d'exécution moyen pour rda6 est 9.370176121592521e-06 secondes.


# 5 shards

In [31]:
queries = [ru1, ru2, ru3, ru6, rda1, rda4, rda5, rda6]

execution_times_5shard = {}

for query in queries:
    times = timeit.repeat(query, repeat=12, number=1)

    times.remove(max(times))
    times.remove(min(times))

    execution_times_5shard[query.__name__] = times

    average_time = sum(times) / len(times)

    print(f"Le temps d'exécution moyen pour {query.__name__} est {average_time} secondes.")

Le temps d'exécution moyen pour ru1 est 0.20991149004548787 secondes.
Le temps d'exécution moyen pour ru2 est 9.749969467520714e-06 secondes.
Le temps d'exécution moyen pour ru3 est 1.1759996414184571e-05 secondes.
Le temps d'exécution moyen pour ru6 est 0.11961727999150754 secondes.
Le temps d'exécution moyen pour rda1 est 0.08920738999731839 secondes.
Le temps d'exécution moyen pour rda4 est 0.6992022800724953 secondes.
Le temps d'exécution moyen pour rda5 est 0.5451671400107443 secondes.
Le temps d'exécution moyen pour rda6 est 1.5190104022622108e-05 secondes.


# 6 shards

In [32]:
queries = [ru1, ru2, ru3, ru6, rda1, rda4, rda5, rda6]

execution_times_6shard = {}

for query in queries:
    times = timeit.repeat(query, repeat=12, number=1)

    times.remove(max(times))
    times.remove(min(times))

    execution_times_6shard[query.__name__] = times

    average_time = sum(times) / len(times)

    print(f"Le temps d'exécution moyen pour {query.__name__} est {average_time} secondes.")

Le temps d'exécution moyen pour ru1 est 0.21108532017096876 secondes.
Le temps d'exécution moyen pour ru2 est 1.3739988207817078e-05 secondes.
Le temps d'exécution moyen pour ru3 est 1.588999293744564e-05 secondes.
Le temps d'exécution moyen pour ru6 est 0.11040642005391418 secondes.
Le temps d'exécution moyen pour rda1 est 0.07599386996589601 secondes.
Le temps d'exécution moyen pour rda4 est 0.6598649200052022 secondes.
Le temps d'exécution moyen pour rda5 est 0.49941287995316086 secondes.
Le temps d'exécution moyen pour rda6 est 9.110011160373687e-06 secondes.


# Sauvegarde

In [1]:
import pickle

In [34]:
with open('dictionaries.pkl', 'wb') as f:
    pickle.dump([execution_times_1shard, execution_times_2shard, execution_times_3shard, 
                 execution_times_4shard, execution_times_5shard, execution_times_6shard], f)

In [2]:
with open('dictionaries.pkl', 'rb') as f:
    execution_times_1shard, execution_times_2shard, execution_times_3shard, execution_times_4shard, execution_times_5shard, execution_times_6shard = pickle.load(f)

# Plot

In [ ]:
import matplotlib.pyplot as plt

queries = ['ru1', 'ru2', 'ru3', 'ru6', 'rda1', 'rda4', 'rda5', 'rda6']

for query in queries:
    plt.plot(execution_times_1shard[query], label='execution_times_1shard')
    plt.plot(execution_times_2shard[query], label='execution_times_2shard')
    plt.plot(execution_times_3shard[query], label='execution_times_3shard')
    plt.plot(execution_times_4shard[query], label='execution_times_4shard')
    plt.plot(execution_times_5shard[query], label='execution_times_5shard')
    plt.plot(execution_times_6shard[query], label='execution_times_6shard')

    plt.title(f'Comparison of {query} execution times on different configurations')
    plt.xlabel('Index')
    plt.ylabel('Values')
    plt.legend()
    plt.show()